In [5]:
!pip install --quiet instructor pydantic openai

In [6]:
from pathlib import Path
import pandas as pd
import sqlite3
import base64
from getpass import getpass
from datetime import date
import instructor
from pydantic import BaseModel
from openai import OpenAI

In [ ]:
openai_key = getpass("OpenAI key: ")

## Czytamy dane klientów

In [21]:
with sqlite3.connect('zad_domowe__clients.db') as conn:
    clients_df = pd.read_sql_query("SELECT * FROM clients", conn)

clients_df.sample(10)

,id,name,email,phone
36,37,Charles Meyer,ashleybowman@example.com,(794)557-0421x64914
59,60,Troy Alexander,nturner@example.net,+1-212-349-3992x430
44,45,John Mitchell,joel50@example.com,478-457-6738x2516
67,68,Carolyn Salazar,mirandatanya@example.com,(547)876-7714x7722
96,97,Olivia Barnes,dpacheco@example.net,9317151015
88,89,Allison Webster,justincurry@example.net,+1-781-673-0341
74,75,Ashley Davis,francisdevon@example.org,564.209.9187
97,98,Tara Cabrera,geraldjames@example.net,+1-984-996-5056
65,66,Walter Payne,geraldserrano@example.com,334-474-5875x05304
99,100,Johnny Franklin,bankslisa@example.org,(626)681-2730x71696


## Odczytujemy dane produktów

In [22]:
products_df = pd.read_csv('zad_domowe__products.csv', sep=None, engine ='python')

products_df.sample(10)

,id,name,price
0,1,telewizor,85
5,6,kamera,92
2,3,smartfon,60
13,14,rolki,11
9,10,pendrive,47
12,13,skateboard,81
6,7,router,48
19,20,spodnie,14
1,2,laptop,47
3,4,tablet,40


## Czytamy dane faktur

In [23]:
DATA_PATH = Path('.')

In [24]:
# Class InvoiceInfoItem
class InvoiceInfoItem(BaseModel):
    description: str
    product_id: int
    quantity: int
    price: float

# TClass InvoiceInfo
class InvoiceInfo(BaseModel):
    company_name: str
    customer_id: int
    customer_name: str
    invoice_number: int
    date: date
    items: list[InvoiceInfoItem]

#Connection with OpenAI
instructor_openai_client = instructor.from_openai(OpenAI(api_key=openai_key))
orders = []
for image_path in sorted(DATA_PATH.glob("zad_domowe__invoice*.png")):
    print(f"Processing {image_path}")
    with open(image_path, "rb") as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')

    response = instructor_openai_client.chat.completions.create(
        model="gpt-4o",
        response_model=InvoiceInfo,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        #Instrukcja dla AI
                        "text": "Pobierz proszę informacje na temat faktury",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{image_data}",
                            "detail": "high"
                        },
                    },
                ],
            },
        ],
    )      
    invoice_info = InvoiceInfo(**response.model_dump())
                                                 
    invoice_data = invoice_info.model_dump()
    for item in invoice_data["items"]:
        order = {
            "company_name": invoice_data["company_name"],
            "customer_id": invoice_data["customer_id"],
            "customer_name": invoice_data["customer_name"],
            "invoice_number": invoice_data["invoice_number"],
            "date": invoice_data["date"],
            "description": item["description"],
            "product_id": item["product_id"],
            "quantity": item["quantity"],
            "price": item["price"],
        }
        orders.append(order)

orders_df = pd.DataFrame(orders)
orders_df

Processing zad_domowe__invoice_1-2ghlei0.png
Processing zad_domowe__invoice_10-zr75ktc.png
Processing zad_domowe__invoice_2-kbskvl0.png
Processing zad_domowe__invoice_3-lq35ebo.png
Processing zad_domowe__invoice_4-q1wt7gh.png
Processing zad_domowe__invoice_5-yph5hub.png
Processing zad_domowe__invoice_6-djmpuya.png
Processing zad_domowe__invoice_7-a73zvn1.png
Processing zad_domowe__invoice_8-jyhggh6.png
Processing zad_domowe__invoice_9-y4r45y1.png


,company_name,customer_id,customer_name,invoice_number,date,description,product_id,quantity,price
0,Firma XYZ,3,Eric Moore,2,2024-05-25,router,7,3,48.0
1,Firma XYZ,3,Eric Moore,2,2024-05-25,narty,17,5,55.0
2,Firma XYZ,92,Valerie Stevens,11,2024-05-02,słuchawki,5,1,40.0
3,Firma XYZ,60,Troy Alexander,3,2024-05-19,ochraniacze,16,4,46.0
4,Firma XYZ,78,William Velazquez,4,2024-03-26,rolki,14,3,11.0
5,Firma XYZ,33,Steven Jackson,5,2024-07-23,pendrive,10,4,47.0
6,Firma XYZ,33,Steven Jackson,5,2024-07-23,laptop,2,2,47.0
7,Firma XYZ,8,Ronald Callahan,6,2024-03-14,słuchawki,5,4,40.0
8,Firma XYZ,8,Ronald Callahan,6,2024-03-14,ochraniacze,16,2,46.0
9,Firma XYZ,4,Laura Beard,7,2024-06-24,telewizor,1,2,85.0


## Teraz łączymy dane

In [27]:
merged_df = orders_df.merge(clients_df, left_on='customer_id', right_on='id', how='left', suffixes=('', '_client'))
merged_df = merged_df.merge(products_df, left_on='product_id', right_on='id', how='left', suffixes=('', '_product'))
final_df = merged_df[['customer_id', 'name', 'email', 'phone', 'product_id', 'name_product', 'quantity', 'price', 'invoice_number', 'date', 'company_name']]
final_df.columns = ['customer_id', 'customer_name', 'email', 'phone', 'product_id', 'product_name', 'quantity', 'unit_price', 'invoice_number', 'date', 'company_name']
final_data = final_df.assign(date=pd.to_datetime(final_df['date']).dt.strftime('%Y-%m-%d'))
final_data.to_csv('final_data.csv', index=False, sep=';', encoding='utf-8')
print("\nFinałowe dane:")
final_data


Finałowe dane:


,customer_id,customer_name,email,phone,product_id,product_name,quantity,unit_price,invoice_number,date,company_name
0,3,Eric Moore,lejacob@example.org,923-681-4646,7,router,3,48.0,2,2024-05-25,Firma XYZ
1,3,Eric Moore,lejacob@example.org,923-681-4646,17,narty,5,55.0,2,2024-05-25,Firma XYZ
2,92,Valerie Stevens,xlowe@example.com,757-943-1579x98346,5,słuchawki,1,40.0,11,2024-05-02,Firma XYZ
3,60,Troy Alexander,nturner@example.net,+1-212-349-3992x430,16,ochraniacze,4,46.0,3,2024-05-19,Firma XYZ
4,78,William Velazquez,lisa25@example.net,(765)602-1572x949,14,rolki,3,11.0,4,2024-03-26,Firma XYZ
5,33,Steven Jackson,jeremy07@example.com,481.258.2434,10,pendrive,4,47.0,5,2024-07-23,Firma XYZ
6,33,Steven Jackson,jeremy07@example.com,481.258.2434,2,laptop,2,47.0,5,2024-07-23,Firma XYZ
7,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,5,słuchawki,4,40.0,6,2024-03-14,Firma XYZ
8,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,16,ochraniacze,2,46.0,6,2024-03-14,Firma XYZ
9,4,Laura Beard,westdominique@example.org,(291)558-0966,1,telewizor,2,85.0,7,2024-06-24,Firma XYZ
